# **Классификация**

Для лабороторной работы используются массив данных "Best Selling Albums of All Time", его можно взять с сайта https://www.kaggle.com/datasets/ciroduro01/best-selling-albums-of-all-time/data?select=best_selling_albums.csv .

In [28]:
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_palette('husl')
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

# Загрузка данных

In [29]:
# Присвоим переменной «url» ссылку на датасет:

url = '/content/best_selling_albums.csv'

# создаем лист с названием столбцов:

cols = ['Album','Artist','Year','Sales_Millions','Decade',"Country", "Genre"]

# используем Pandas read_csv():

dataset = pd.read_csv(url, names = cols)

# Анализ данных

In [30]:
# размер датасета, 50 записей, 7 столбцов

dataset.shape

(50, 7)

In [31]:
# отображаем первые 10 строк датасета

dataset.head(10)

,Album,Artist,Year,Sales_Millions,Decade,Country,Genre
0,Album,Artist,Year,Sales_Millions,Decade,Country,Genre
1,Thriller,Michael Jackson,1982,70,1980s,USA,Pop
2,Back in Black,AC/DC,1980,50,1980s,Australia,Rock
3,The Dark Side of the Moon,Pink Floyd,1973,45,1970s,UK,Rock
4,The Bodyguard,Whitney Houston,1992,45,1990s,USA,Pop
5,Bat Out of Hell,Meat Loaf,1977,43,1970s,USA,Rock
6,Their Greatest Hits (1971–1975),Eagles,1976,42,1970s,USA,Rock
7,Hotel California,Eagles,1976,42,1970s,USA,Rock
8,Come On Over,Shania Twain,1997,40,1990s,Canada,Country
9,Rumours,Fleetwood Mac,1977,40,1970s,UK,Rock


In [32]:
import mpl_toolkits.mplot3d  # noqa: F401


from sklearn import datasets
from sklearn.cluster import KMeans

In [33]:
# Метод Pandas info()  выводит очень много информации о наборе данных - количество записей,
# количество ненулевых значений в каждом столбце, тип данных в столбце

dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Album           50 non-null     object
 1   Artist          50 non-null     object
 2   Year            50 non-null     object
 3   Sales_Millions  50 non-null     object
 4   Decade          50 non-null     object
 5   Country         50 non-null     object
 6   Genre           50 non-null     object
dtypes: object(7)
memory usage: 2.9+ KB


In [34]:
# Pandas describe() используется для просмотра  основной статистической информации, таких как процентиль, среднее,
# стандартное отклонение и т.д. фрейма данных или ряда числовых значений:

dataset.describe()

,Album,Artist,Year,Sales_Millions,Decade,Country,Genre
count,50,50,50,50,50,50,50
unique,50,39,32,21,8,6,11
top,Album,Michael Jackson,1987,22,1980s,USA,Pop
freq,1,3,6,6,13,30,21


In [35]:
dataset['Genre'].unique()

array(['Genre', 'Pop', 'Rock', 'Country', 'Disco', 'Metal', 'Grunge',
       'Folk', 'R&B', 'Soul', 'Hip Hop'], dtype=object)

In [36]:
# Теперь проверим количество строк, которое принадлежит каждому классу:

dataset['Genre'].value_counts()

,count
Genre,
Pop,21
Rock,17
Country,2
Hip Hop,2
Metal,2
Genre,1
Disco,1
Grunge,1
Folk,1


Видим, что набор данных несбалансированный.

In [37]:
train = pd.read_csv('best_selling_albums.csv')

In [38]:
train.head()

,Album,Artist,Year,Sales_Millions,Decade,Country,Genre
0,Thriller,Michael Jackson,1982,70,1980s,USA,Pop
1,Back in Black,AC/DC,1980,50,1980s,Australia,Rock
2,The Dark Side of the Moon,Pink Floyd,1973,45,1970s,UK,Rock
3,The Bodyguard,Whitney Houston,1992,45,1990s,USA,Pop
4,Bat Out of Hell,Meat Loaf,1977,43,1970s,USA,Rock


In [39]:
train.describe()

,Year,Sales_Millions
count,49.000000,49.000000
mean,1988.306122,30.734694
std,13.731357,9.346959
min,1957.000000,20.000000
25%,1977.000000,24.000000
50%,1987.000000,28.000000
75%,1999.000000,33.000000
max,2017.000000,70.000000


In [40]:
from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()
train[['Album','Artist','Year','Sales_Millions','Genre','Country', "Decade"]] = enc.fit_transform(train[['Album','Artist','Year','Sales_Millions','Genre','Country', "Decade"]])
train.head()

,Album,Artist,Year,Sales_Millions,Decade,Country,Genre
0,46.0,24.0,11.0,19.0,3.0,4.0,6.0
1,5.0,0.0,10.0,18.0,3.0,0.0,8.0
2,41.0,26.0,5.0,17.0,2.0,3.0,8.0
3,40.0,37.0,17.0,17.0,4.0,4.0,6.0
4,8.0,22.0,7.0,16.0,2.0,4.0,8.0


In [41]:
enc.categories_

[array(['1', '1989', '21', 'Abbey Road', 'Appetite for Destruction',
        'Back in Black', 'Back to Black', 'Bad', 'Bat Out of Hell',
        'Born in the U.S.A.', 'Bridge Over Troubled Water',
        'Brothers in Arms', 'Come On Over', 'Confessions', 'Dangerous',
        'Dirty Dancing Soundtrack', 'Divide', 'Elvis’ Christmas Album',
        'Faith', 'Fearless', 'FutureSex/LoveSounds',
        'Goodbye Yellow Brick Road', 'Grease Soundtrack',
        'Hotel California', 'Hybrid Theory', 'Jagged Little Pill',
        'Led Zeppelin IV', 'Like a Virgin', 'Metallica (Black Album)',
        'Millennium', 'Nevermind', 'Oops!... I Did It Again',
        'Purple Rain', 'Recovery', 'Rumours', 'Saturday Night Fever',
        'Sgt. Pepper’s Lonely Hearts Club Band', 'Spice', 'Supernatural',
        'Tapestry', 'The Bodyguard', 'The Dark Side of the Moon',
        'The Eminem Show', 'The Joshua Tree', 'The Wall',
        'Their Greatest Hits (1971–1975)', 'Thriller', 'True Blue',
        'Whi

# Построение модели

## Разделение набора данных

Y  - целевая переменная, которую предсказываем. Мы ищем зависимость этой переменной от соответствующих фич.  В нашем случае это столбец class.

В X складываем весь набор данных без целевой переменной.

In [42]:
X = train.drop(['Genre'], axis=1)
y = train['Genre']
print(f'X shape: {X.shape} | y shape: {y.shape} ')

X shape: (49, 6) | y shape: (49,) 


X имеет 49 строк и 6 столбца, тогда как Y имеет 49 строк и только один столбец.


## Train Test разделение

Разделяем наш набор данных на обучающий (train) и тестовый (test) с помощью train_test_split(), мы берем 80% данных для обучения нашей модели и 20% оставляем в качестве проверочного набора данных:

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=10)

## Обучение и выбор оптимальной модели

Далее попробуем следующие алгоритмы:

Logistic Regression (LR) </br>
Linear Discriminant Analysis (LDA)</br>
K-Nearest Neighbors (KNN)</br>
Classification and Regression Trees (CART)</br>
Gaussian Naive Bayes (NB)</br>
Support Vector Machines (SVM)</br>

In [19]:
# создаем лист для тех моделей, которые будем изучать
models = []
models.append(('LR', LogisticRegression(solver='lbfgs', max_iter=1000)))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVC', SVC(gamma='auto')))

# оцениваем их метрики
results = []
model_names = []
for name, model in models:
  kfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
  cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring='accuracy')
  results.append(cv_results)
  model_names.append(name)
  print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))

/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/mod

LR: 0.383333 (0.197906)
LDA: 0.433333 (0.246644)
KNN: 0.375000 (0.167705)
CART: 0.358333 (0.226844)
NB: 0.408333 (0.225000)
SVC: 0.433333 (0.104083)


/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


Получившиеся данные:</br>
LR: 0.383333 (0.197906)</br>
LDA: 0.433333 (0.246644)</br>
KNN: 0.375000 (0.167705)</br>
CART: 0.333333 (0.250000)</br>
NB: 0.408333 (0.225000)</br>
SVC: 0.433333 (0.104083)</br>

SVC показал лучшиу метрики среди всех алгоритмов.
Далее работаем с SVC.

Давайте обучим модель LDA на нашем обучающем наборе и сделаем прогноз на тестовом наборе на следующем шаге.

## Обучение конкретной модели
Определяем нашу модель SVC
После этого подгоняем/обучаем модель на X_train и Y_train с помощью метода .fit().
Затем мы делаем прогноз на X_test с помощью метода .predict().

In [20]:
sk_lda = LinearDiscriminantAnalysis(solver='eigen')
sk_lda.fit(X_train, y_train)
sk_lda_pred_res = sk_lda.predict(X_test)
sk_transformed = sk_lda.transform(X_train)
sk_lda_accuracy = accuracy_score(y_test, sk_lda_pred_res)

print(f'sk LDA accuracy: {sk_lda_accuracy}')
print(f'sk LDA prediction: {sk_lda_pred_res}')
print('sk transformed features', sk_transformed[:5].T, sep='\n')

sk LDA accuracy: 0.4
sk LDA prediction: [7. 6. 5. 4. 6. 6. 6. 6. 8. 8.]
sk transformed features
[[-7.20951362 -3.22403167 -6.03138986 -5.83747757 -3.22180458]
 [-1.16187956 -2.69103639 -0.98271113  0.64317952 -3.72490707]
 [-1.27567293 -2.66912455 -1.22120337 -2.07784184 -2.78352727]
 [ 2.91446138  5.77134115  1.70092817  3.33654292  4.00562685]
 [ 3.33045605  3.2035495   2.99540626  2.44427883  3.10530339]
 [ 1.8642751   2.16815936  2.50789415  3.72753159  2.30374317]]


/usr/local/lib/python3.12/dist-packages/sklearn/covariance/_empirical_covariance.py:102: UserWarning: Only one sample available. You may want to reshape your data array
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/covariance/_empirical_covariance.py:102: UserWarning: Only one sample available. You may want to reshape your data array
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/covariance/_empirical_covariance.py:102: UserWarning: Only one sample available. You may want to reshape your data array
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/covariance/_empirical_covariance.py:102: UserWarning: Only one sample available. You may want to reshape your data array
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/covariance/_empirical_covariance.py:102: UserWarning: Only one sample available. You may want to reshape your data array
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/covariance/_empirical_

Получившиеся данные: </br>
sk LDA accuracy: 0.4</br>
sk LDA prediction: [7. 6. 5. 4. 6. 6. 6. 6. 8. 8.]</br>
sk transformed features</br>
[[-7.20951362 -3.22403167 -6.03138986 -5.83747757 -3.22180458]</br>
 [-1.16187956 -2.69103639 -0.98271113  0.64317952 -3.72490707]</br>
 [-1.27567293 -2.66912455 -1.22120337 -2.07784184 -2.78352727]</br>
 [ 2.91446138  5.77134115  1.70092817  3.33654292  4.00562685]</br>
 [ 3.33045605  3.2035495   2.99540626  2.44427883  3.10530339]</br>
 [ 1.8642751   2.16815936  2.50789415  3.72753159  2.30374317]]</br>